In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import RFE
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
#Read data :- 
data=pd.read_csv('/kaggle/input/life-expectancy-who/Life Expectancy Data.csv')
data.head()
print(data.info())
print(data.shape)
print(data.describe())

In [ ]:
# Perform EDA:-
data.isna().sum()
#we can see many columns do come with missing data let us deal with them one by one

# Remove life expectancy where  ever it is zero as these are only one years data in the database for a country
data['Life expectancy '].fillna(0,inplace=True)
data=data[data['Life expectancy '] != 0]


# A lot of countries do not have population details we can remove these rows.
data['Population'].fillna(0,inplace=True)
data=data[data['Population'] != 0]
data.isna().sum()

In [ ]:
## Missing Value Interpretation :

#In order to find the Missing values let us take average of individual columns for that oarticulat country
import numpy as np
co=data.copy()
co[co== 0] = np.nan
data_mean=pd.pivot_table(co,index=['Country'],values=['Life expectancy ','Hepatitis B','Adult Mortality','Alcohol',' BMI ','Total expenditure','Population','Income composition of resources','GDP',' thinness 5-9 years','Schooling'],
                         aggfunc=np.nanmean)
data_mean=pd.DataFrame(data_mean.to_records())
data_mean.fillna(0,inplace=True)
data.isna().sum()
data['Alcohol'].fillna(0,inplace=True)



#from progressbar import ProgressBar
#Alcohall fill with nanmean
#Pbar=ProgressBar()

for i in data.index:
    Cou=data.loc[i,'Country']
    Alco =data.loc[i,'Alcohol']
    S=data_mean[data_mean['Country'] == Cou]
    M=S['Alcohol'].values[0]
    S.index
    if Alco == 0 :
        data.loc[i,'Alcohol'] = M
        
#Hepa fill with nanmean
        
data['Hepatitis B'].fillna(0,inplace=True)
for i in data.index:
    Cou=data.loc[i,'Country']
    Alco =data.loc[i,'Hepatitis B']
    S=data_mean[data_mean['Country'] == Cou]
    M=S['Hepatitis B'].values[0]
    S.index
    if Alco == 0 :
        data.loc[i,'Hepatitis B'] = M
data.isna().sum() 


#Population has no relation to Life expectancy so we can remove that column
del data['Population']
data.isna().sum()

data['Total expenditure'].fillna(0,inplace=True)


for i in data.index:
    Cou=data.loc[i,'Country']
    Alco =data.loc[i,'Total expenditure']
    S=data_mean[data_mean['Country'] == Cou]
    M=S['Total expenditure'].values[0]
    S.index
    if Alco == 0 :
        data.loc[i,'Total expenditure'] = M
data.isna().sum()        


data[' BMI '].fillna(0,inplace=True)


for i in data.index:
    Cou=data.loc[i,'Country']
    Alco =data.loc[i,' BMI ']
    S=data_mean[data_mean['Country'] == Cou]
    M=S[' BMI '].values[0]
    S.index
    if Alco == 0 :
        data.loc[i,' BMI '] = M
data.isna().sum()    






# thin ness we cannot averge it out so let us remove where ever it is 0, also delete Diphtheria where ever it zero we cannot calculate avergae  
data[' thinness  1-19 years'].fillna(0,inplace=True)
data=data[data[' thinness  1-19 years'] != 0]
data['Diphtheria '].fillna(0,inplace=True)
data=data[data['Diphtheria '] != 0]
data.isna().sum()


#data.GDP.fillna(0,inplace=True)
        
data['GDP'].fillna(0,inplace=True)

for i in data.index:
    Cou=data.loc[i,'Country']
    Alco =data.loc[i,'GDP']
    S=data_mean[data_mean['Country'] == Cou]
    M=S['GDP'].values[0]
    S.index
    if Alco == 0 :
        data.loc[i,'GDP'] = M        
        
data.isna().sum()
#Finally there are no Missing values now.
data.info()

In [ ]:
## Get Dummies for Categorical Column : 
sec=pd.get_dummies(data['Status'])
sec
backdata=data.copy()
data=pd.concat([data,sec],axis=1)
data=data.drop(['Country','Year','Status'],axis=1)
data.head()


In [ ]:
# Test Train split :-
df_train,df_test=train_test_split(data,train_size=0.7,test_size=0.3,random_state=100)
print(df_train.shape)
print(df_test.shape)

In [ ]:
#Standard Scalar:-
scaler=MinMaxScaler()
var=['Life expectancy ','Adult Mortality','infant deaths','Alcohol','percentage expenditure','Hepatitis B','Measles ',' BMI ','under-five deaths ','Polio','Total expenditure','Diphtheria ',' HIV/AIDS','GDP',' thinness  1-19 years',' thinness 5-9 years','Income composition of resources','Schooling']
df_train[var]=scaler.fit_transform(df_train[var])
df_train.head()
# Split dependant and Independant data set
y_train = df_train.pop('Life expectancy ')
X_train=df_train


In [ ]:
#Run RFE :- 
# RFE to get the most significant features
lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, 10)
rfe = rfe.fit(X_train, y_train)
list(zip(X_train.columns, rfe.support_, rfe.ranking_))
cols = X_train.columns[rfe.support_]
cols

In [ ]:

X_train_rfe = X_train[cols]
X_train_rfe = sm.add_constant(X_train_rfe)
vif = pd.DataFrame()
vif['Features'] = X_train_rfe.columns
vif['VIF'] = [variance_inflation_factor(X_train_rfe.values, i) for i in range(X_train_rfe.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending=False) 
vif

In [ ]:
lm=sm.OLS(y_train,X_train_rfe).fit()
print(lm.summary())

In [ ]:
cols = cols.drop('GDP')
X_train_rfe = X_train[cols]
X_train_rfe = sm.add_constant(X_train_rfe)
lm = sm.OLS(y_train, X_train_rfe).fit()
print(lm.summary())


In [ ]:
X_train_rfe = X_train[cols]
X_train_rfe = sm.add_constant(X_train_rfe)
vif = pd.DataFrame()
vif['Features'] = X_train_rfe.columns
vif['VIF'] = [variance_inflation_factor(X_train_rfe.values, i) for i in range(X_train_rfe.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending=False) 
vif

In [ ]:
cols = cols.drop('under-five deaths ')
X_train_rfe = X_train[cols]
X_train_rfe = sm.add_constant(X_train_rfe)
lm = sm.OLS(y_train, X_train_rfe).fit()
print(lm.summary())

In [ ]:
cols = cols.drop('infant deaths')
X_train_rfe = X_train[cols]
X_train_rfe = sm.add_constant(X_train_rfe)
lm = sm.OLS(y_train, X_train_rfe).fit()
print(lm.summary())

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X_train_rfe.columns
vif['VIF'] = [variance_inflation_factor(X_train_rfe.values, i) for i in range(X_train_rfe.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending=False) 
vif
# VIF is within the range which is less than 5

In [ ]:
y_train_pred = lm.predict(X_train_rfe)
res = y_train - y_train_pred
sns.distplot(res)
# Residual mean falling 0 and is a normal distribution so model is significant  

###  Current :- R-squared: 83.7.%
### Adjusted R-squared: 83.6%
### P values are equal to 0
### infant deaths, percentage expenditure , BMI ,Diphtheria Schooling having postive correlations
### under-five deaths, Adult Mortality,HIV/AIDS having negative impact 

In [ ]:
# Run on Test:-
df_test[var] = scaler.transform(df_test[var])
df_test.head()
y_test=df_test.pop('Life expectancy ')
X_test = df_test
X_test
X_test_rfe = X_test[cols]
X_test_rfe.head()
X_test_rfe.info()

In [ ]:
X_test_rfe = sm.add_constant(X_test_rfe)
X_test_rfe.head()
#Accuracy :-
y_test_pred = lm.predict(X_test_rfe)
r2_score(y_true=y_test, y_pred=y_test_pred)

## Model accuracy is good 84.37% 
##### We can see Where in countries where HIV/AIDS and Adult Mortality are more such countries has low life expectancy
##### And in the countries where schooling numbers,Income composition of resources, BMI and Diphtheria and also percentage expenditure are more we can see they genearally have more life expectancy.


## So if Countries wants to increase life expectancy of the people they realy should be focusing to have more kids covered in schooling and they need to increase the income of the people and Increase people percentage expendeture mainly and also we need to increase the awareness on HIV/AIDS and also need to make sure we maintain low Adult mortality rate.
